6 comb

* vectorization: 
    a. bow
    b. tfidf
    * feature selection
        a. chi2
        b. f-test
        c. mi
        models:
            # logistic: 1008 comb
            # svm: 2688
            # rf: 8000
            
            
total: 82560 combs
lots of 5 cross validation = g


In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
import re
from data_cleaning import *

# 1. Preprocessing

In [2]:
# read data
train_data = pd.read_csv("Train.csv", sep=',')
test_data = pd.read_csv("Test.csv", sep=',')

In [3]:
#separating instance and label for Train
X_train_raw = [x[0] for x in train_data[['text']].values]
Y_train = [x[0] for x in train_data[['sentiment']].values]
X_test_raw = [x[0] for x in test_data[['text']].values]

#### (1). data cleaning

In [ ]:
# 1. data cleaning (optional)
X_train_need_to_clean = pd.DataFrame(X_train_raw)
X_test_need_to_clean = pd.DataFrame(X_test_raw)

# remove url, # and @
X_train_need_to_clean.replace("\b*https?:\S*", '', regex=True, inplace=True)
X_train_need_to_clean.replace("\b*@\S*", '', regex=True, inplace=True)
X_train_need_to_clean.replace("\b*#\S*", '', regex=True, inplace=True)
X_test_need_to_clean.replace("\b*https?:\S*", '', regex=True, inplace=True)
X_test_need_to_clean.replace("\b*@\S*", '', regex=True, inplace=True)
X_test_need_to_clean.replace("\b*#\S*", '', regex=True, inplace=True)

for i in range(X_train_need_to_clean.shape[0]):
    X_train_need_to_clean.loc[i, 0] = ' '.join(text_preprocessing(X_train_need_to_clean.loc[i, 0], remove_html=False))

for i in range(X_test_need_to_clean.shape[0]):
    X_test_need_to_clean.loc[i, 0] = ' '.join(text_preprocessing(X_test_need_to_clean.loc[i, 0], remove_html=False))    

X_train_need_to_clean.replace('[^\w\s]','', regex=True, inplace=True)
X_test_need_to_clean.replace('[^\w\s]','', regex=True, inplace=True)
    
    
X_train_clean = [x[0] for x in X_train_need_to_clean[[0]].values]
X_test_clean = [x[0] for x in X_test_need_to_clean[[0]].values]

#### (2). vectorization (transformation)

In [ ]:
# bag of words
# countvectorizer
BoW_vectorizer = CountVectorizer(ngram_range=(2,2))

X_train_BoW = BoW_vectorizer.fit_transform(X_train_clean)
X_test_BoW = BoW_vectorizer.transform(X_test_clean)

X_train_tranformed, X_test_transformed = X_train_BoW, X_test_BoW

In [ ]:
#TFIDF
tfidf_vectorizer = TfidfVectorizer(ngram_range=(2,2))
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_clean)


X_test_tfidf = tfidf_vectorizer.transform(X_test_raw)
X_train_tranformed, X_test_transformed = X_train_tfidf, X_test_tfidf

### comparison between data with preprocessing and without preprocessing

In [13]:
CountVectorizer(analyzer='word', ngram_range=(2,2)).fit_transform(X_train_raw).shape

(21802, 204657)

In [14]:
X_train_tranformed.shape

(21802, 127482)

# 2. feature selection

### type of selector (choose one of them)

In [ ]:
# chi2     
selector = SelectKBest(chi2,k=10000)
X_train_new = selector.fit_transform(X_train_tranformed,Y_train)
X_test_new = selector.transform(X_test_transformed)

In [ ]:
# f_classifs
selector = SelectKBest(f_classif,k=5000)
X_train_new = selector.fit_transform(X_train_tranformed,Y_train)
X_test_new = selector.transform(X_test_transformed)

In [ ]:
# mutual_info_classif
selector = SelectKBest(mutual_info_classif,k=10000)
X_train_new = selector.fit_transform(X_train_tranformed,Y_train)
X_test_new = selector.transform(X_test_transformed)

# 3. training

### split data for validation

In [ ]:
train_size = X_train_tranformed.shape[0]
test_size = X_test_transformed.shape[0]
## random hold out
ts = test_size/train_size
X_train_s, X_validation, y_train_s, y_validation = train_test_split(X_train_new,Y_train, test_size=ts)

In [ ]:
cv = round(X_train_s.shape[0] / X_validation.shape[0])

#### Prototype selection

In [51]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_resample(X_train_s, y_train_s)
X_train_s, y_train_s = X_resampled, y_resampled

In [52]:
resampled_labels = pd.DataFrame(y_resampled)
resampled_labels.value_counts()

negative    2668
neutral     2668
positive    2668
dtype: int64

#### Prototype generation

In [9]:
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=0)
X_resampled, y_resampled = cc.fit_resample(X_train_s, y_train_s)
X_train_s, y_train_s = X_resampled, y_resampled

KeyboardInterrupt: 

In [ ]:
resampled_labels = pd.DataFrame(y_resampled)
resampled_labels.value_counts()

In [ ]:
cv = round(X_train_s.shape[0] / X_validation.shape[0])

## base model: 0R

In [ ]:
clf = DummyClassifier(strategy='most_frequent')
basemodel = clf.fit(X_train_raw, Y_train)
print("base model score: ", basemodel.score(X_train_raw, Y_train))

## logistic regression

### choose of hyperparameter

In [ ]:
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
from sklearn.linear_model import LogisticRegression

hyper = {
    'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'],
    'penalty': ['l1', 'l2', 'none', 'elasticnet'],
    'C': [1e-3, 1e-2, 1e-1, 1, 10, 100, 1000],
    'max_iter': [100, 200, 300],
    'multi_class': ['auto', 'ovr', 'multinomial']
}

##### randomised method (less computation)

In [ ]:
search_logi = RandomizedSearchCV(LogisticRegression(),hyper, scoring='accuracy', cv=cv, n_iter=101)
logi_result = search_logi.fit(X_train_s, y_train_s)

##### grid search

In [ ]:
search_logi = GridSearchCV(LogisticRegression(),hyper, scoring='accuracy', cv=cv)
logi_result = search_logi.fit(X_train_s, y_train_s)

##### evaluation

In [ ]:
pd.DataFrame(logi_result.cv_results_)
print(logi_result.best_params_)
print(logi_result.best_score_)

### SVM

In [ ]:
from sklearn.svm import SVC

svm_hyper = {
    'degree': [3, 5, 10, 15],
    'gamma': [1,0.1,0.01,0.001],
    'C': [1e-3, 1e-2, 1e-1, 1, 10, 100, 1000],
    'kernel': ['linear', 'poly', 'rbf', 'sigmod'],
    'max_iter': [100, 500, 1000],
    'decision_function_shape': ['ovo', 'ovr']
}

In [ ]:
# random method
search_svm = RandomizedSearchCV(SVC(), svm_hyper, scoring='accuracy', cv=cv, n_iter=269)
svm_result = search_svm.fit(X_train_s, y_train_s)

In [ ]:
# grid method
search_svm = GridSearchCV(SVC(), svm_hyper, scoring='accuracy', cv=cv)
svm_result = search_svm.fit(X_train_s, y_train_s)

In [ ]:
pd.DataFrame(svm_result.cv_results_)
print(svm_result.best_params_)
print(svm_result.best_score_)

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_hyper = {
    'n_estimators': [90, 100, 115 , 130],
    'criterion': ['gini', 'entropy'],
    'max_depth': range(2,20,2),
    'min_samples_leaf': range(1,10,1),
    'min_samples_split': range(2,10,2),
    'max_features': ['auto', 'log2']
}

In [ ]:
# random method
search_svm = RandomizedSearchCV(RandomForestClassifier(), rf_hyper, scoring='accuracy', cv=cv, n_iter=800)
rf_result = search_svm.fit(X_train_s, y_train_s)

In [ ]:
# grid method 
search_svm = GridSearchCV(RandomForestClassifier(), rf_hyper, scoring='accuracy', cv=cv)
rf_result = search_svm.fit(X_train_s, y_train_s)

In [ ]:
pd.DataFrame(rf_result.cv_results_)
print(rf_result.best_params_)
print(rf_result.best_score_)

### Stacking

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.tree import DecisionTreeClassifier

rf_model = RandomForestClassifier(criterion=rf_result.best_params_['criterion'], max_depth=rf_result.best_params_['max_depth'],
                                  max_features='log2', min_samples_leaf=rf_result.best_params_['min_samples_leaf'], 
                                  min_samples_split=rf_result.best_params_['min_samples_split'], n_estimators=rf_result.best_params_['n_estimators'],
                                  random_state=0).fit(X_train_s, y_train_s)

svm_model = SVC(degree=svm_result.best_params_['degree'], gamma=svm_result.best_params_['gamma'] ,C=svm_result.best_params_['C'], 
                kernel=svm_result.best_params_['kernel'], max_iter=svm_result.best_params_['max_iter'], 
                decision_function_shape=svm_result.best_params_['decision_function_shape']).fit(X_train_s, y_train_s)

logi_model = LogisticRegression(solver=logi_result.best_params_['solver'], penalty=logi_result.best_params_['penalty'],
                                C=logi_result.best_params_['C'], max_iter=logi_result.best_params_['max_iter'],
                                multi_class=logi_result.best_params_['multi_class']).fit(X_train_s, y_train_s)

estimators = [('rf', rf_model),('svr', svm_model), ('log', logi_model)]

In [ ]:
dt_stacking = StackingClassifier(estimators=estimators, final_estimator=DecisionTreeClassifier()).fit(X_train_s, y_train_s)
dt_stacking.score(X_validation, y_validation)

In [ ]:
svm_stacking = StackingClassifier(estimators=estimators, final_estimator=SVC()).fit(X_train_s, y_train_s)
svm_stacking.score(X_validation, y_validation)

In [ ]:
logi_stacking = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(max_iter=200)).fit(X_train_s, y_train_s)
logi_stacking.score(X_validation, y_validation)

# 4. Evaluation

### score

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

#### 0R

In [ ]:
X_train_s.shape

In [ ]:
X_validation.shape

In [ ]:
# cv =round(X_train_s.shape[0]/X_validation.shape[0])
cv = 5


# basestic
print("basemodel: 0R")

# validation acc
vc = np.mean(cross_val_score(basemodel,X_validation, y_validation, cv=cv))
print("Validation accuracy: ", vc)

# predictions for test data
base_label = basemodel.predict(X_test_new)

base_validation_label = basemodel.predict(X_validation)

# macro metrics
base_precision_m = precision_score(y_validation,base_validation_label, average = 'macro')
print('macro precision is ', base_precision_m)

base_recall_m = recall_score(y_validation,base_validation_label, average = 'macro')
print('macro recall is ', base_recall_m)

# f1
base_f1_m = f1_score(y_validation,base_validation_label, average = 'macro')
print('macro f1 is ', base_f1_m)


# weighted metrics
base_precision_w = precision_score(y_validation,base_validation_label, average = 'weighted')
print('weighted precision is ', base_precision_w)

base_recall_w = recall_score(y_validation,base_validation_label, average = 'weighted')
print('weighted recall is ', base_recall_w)

# f1
base_f1_w = f1_score(y_validation,base_validation_label, average = 'weighted')
print('weighted f1 is ', base_f1_w)

#### logistic regression

In [ ]:
# logistic
print("Logistic Regression")

# validation acc
vc = np.mean(cross_val_score(logi_model,X_validation, y_validation, cv=cv))
print("Validation accuracy: ", vc)

# predictions for test data
logi_label = logi_model.predict(X_test_new)

logi_validation_label = logi_model.predict(X_validation)

# macro metrics
logi_precision_m = precision_score(y_validation,logi_validation_label, average = 'macro')
print('macro precision is ', logi_precision_m)

logi_recall_m = recall_score(y_validation,logi_validation_label, average = 'macro')
print('macro recall is ', logi_recall_m)

# f1
logi_f1_m = f1_score(y_validation,logi_validation_label, average = 'macro')
print('macro f1 is ', logi_f1_m)


# weighted metrics
logi_precision_w = precision_score(y_validation,logi_validation_label, average = 'weighted')
print('weighted precision is ', logi_precision_w)

logi_recall_w = recall_score(y_validation,logi_validation_label, average = 'weighted')
print('weighted recall is ', logi_recall_w)

# f1
logi_f1_w = f1_score(y_validation,logi_validation_label, average = 'weighted')
print('weighted f1 is ', logi_f1_w)

#### SVM

In [ ]:
# svm
print("SVM")

# validation acc
vc = np.mean(cross_val_score(svm_model,X_validation, y_validation, cv=cv))
print("Validation accuracy: ", vc)

# predictions for test data
svm_label = svm_model.predict(X_test_new)

svm_validation_label = svm_model.predict(X_validation)

# macro metrics
svm_precision_m = precision_score(y_validation,svm_validation_label, average = 'macro')
print('macro precision is ', svm_precision_m)

svm_recall_m = recall_score(y_validation,svm_validation_label, average = 'macro')
print('macro recall is ', svm_recall_m)

# f1
svm_f1_m = f1_score(y_validation,svm_validation_label, average = 'macro')
print('macro f1 is ', svm_f1_m)


# weighted metrics
svm_precision_w = precision_score(y_validation,svm_validation_label, average = 'weighted')
print('weighted precision is ', svm_precision_w)

svm_recall_w = recall_score(y_validation,svm_validation_label, average = 'weighted')
print('weighted recall is ', svm_recall_w)

# f1
svm_f1_w = f1_score(y_validation,svm_validation_label, average = 'weighted')
print('weighted f1 is ', svm_f1_w)

#### random forest

In [ ]:
# random forest
print("random forest")

# validation acc
vc = np.mean(cross_val_score(rf_model,X_validation, y_validation, cv=cv))
print("Validation accuracy: ", vc)

# predictions for test data
rf_label = rf_model.predict(X_test_new)

rf_validation_label = rf_model.predict(X_validation)

# macro metrics
rf_precision_m = precision_score(y_validation,rf_validation_label, average = 'macro')
print('macro precision is ', rf_precision_m)

rf_recall_m = recall_score(y_validation,rf_validation_label, average = 'macro')
print('macro recall is ', rf_recall_m)

# f1
rf_f1_m = f1_score(y_validation,rf_validation_label, average = 'macro')
print('macro f1 is ', rf_f1_m)


# weighted metrics
rf_precision_w = precision_score(y_validation,rf_validation_label, average = 'weighted')
print('weighted precision is ', rf_precision_w)

rf_recall_w = recall_score(y_validation,rf_validation_label, average = 'weighted')
print('weighted recall is ', rf_recall_w)

# f1
rf_f1_w = f1_score(y_validation,rf_validation_label, average = 'weighted')
print('weighted f1 is ', rf_f1_w)

#### stacking

In [ ]:
# svm_stacking
print("svm_stacking")

# validation acc
vc = np.mean(cross_val_score(svm_stacking,X_validation, y_validation, cv=cv))
print("Validation accuracy: ", vc)

# predictions for test data
ss_label = svm_stacking.predict(X_test_new)

ss_validation_label = svm_stacking.predict(X_validation)

# macro metrics
ss_precision_m = precision_score(y_validation,ss_validation_label, average = 'macro')
print('macro precision is ', ss_precision_m)

ss_recall_m = recall_score(y_validation,ss_validation_label, average = 'macro')
print('macro recall is ', ss_recall_m)

# f1
ss_f1_m = f1_score(y_validation,ss_validation_label, average = 'macro')
print('macro f1 is ', ss_f1_m)


# weighted metrics
ss_precision_w = precision_score(y_validation,ss_validation_label, average = 'weighted')
print('weighted precision is ', ss_precision_w)

ss_recall_w = recall_score(y_validation,ss_validation_label, average = 'weighted')
print('weighted recall is ', ss_recall_w)

# f1
ss_f1_w = f1_score(y_validation,ss_validation_label, average = 'weighted')
print('weighted f1 is ', ss_f1_w)

In [ ]:
# dt_stacking
print("dt_stacking")

# validation acc
vc = np.mean(cross_val_score(dt_stacking,X_validation, y_validation, cv=cv))
print("Validation accuracy: ", vc)

# predictions for test data
dts_label = dt_stacking.predict(X_test_new)

dts_validation_label = dt_stacking.predict(X_validation)

# macro metrics
dts_precision_m = precision_score(y_validation,dts_validation_label, average = 'macro')
print('macro precision is ', dts_precision_m)

dts_recall_m = recall_score(y_validation,dts_validation_label, average = 'macro')
print('macro recall is ', dts_recall_m)

# f1
dts_f1_m = f1_score(y_validation,dts_validation_label, average = 'macro')
print('macro f1 is ', dts_f1_m)


# weighted metrics
dts_precision_w = precision_score(y_validation,dts_validation_label, average = 'weighted')
print('weighted precision is ', dts_precision_w)

dts_recall_w = recall_score(y_validation,dts_validation_label, average = 'weighted')
print('weighted recall is ', dts_recall_w)

# f1
dts_f1_w = f1_score(y_validation,dts_validation_label, average = 'weighted')
print('weighted f1 is ', dts_f1_w)

In [ ]:
# logi_stacking
print("logi_stacking")

# validation acc
vc = np.mean(cross_val_score(logi_stacking,X_validation, y_validation, cv=cv))
print("Validation accuracy: ", vc)

# predictions for test data
logi_s_label = logi_stacking.predict(X_test_new)

logi_s_validation_label = logi_stacking.predict(X_validation)

# macro metrics
logi_s_precision_m = precision_score(y_validation,logi_s_validation_label, average = 'macro')
print('macro precision is ', logi_s_precision_m)

logi_s_recall_m = recall_score(y_validation,logi_s_validation_label, average = 'macro')
print('macro recall is ', logi_s_recall_m)

# f1
logi_s_f1_m = f1_score(y_validation,logi_s_validation_label, average = 'macro')
print('macro f1 is ', logi_s_f1_m)


# weighted metrics
logi_s_precision_w = precision_score(y_validation,logi_s_validation_label, average = 'weighted')
print('weighted precision is ', logi_s_precision_w)

logi_s_recall_w = recall_score(y_validation,logi_s_validation_label, average = 'weighted')
print('weighted recall is ', logi_s_recall_w)

# f1
logi_s_f1_w = f1_score(y_validation,logi_s_validation_label, average = 'weighted')
print('weighted f1 is ', logi_s_f1_w)

# 5. Error Analysis

In [ ]:
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
# logistic regression
plot_confusion_matrix(logi_model, X_validation, y_validation)
plt.show()

In [ ]:
# svm
plot_confusion_matrix(svm_model, X_validation, y_validation)
plt.show()

In [ ]:
# random forest
plot_confusion_matrix(rf_model, X_validation, y_validation)
plt.show()

In [ ]:
# svm_stacking
plot_confusion_matrix(svm_stacking, X_validation, y_validation)
plt.show()


In [ ]:
# dt_stacking
plot_confusion_matrix(dt_stacking, X_validation, y_validation)
plt.show()

In [ ]:
# logi_stacking
plot_confusion_matrix(logi_stacking, X_validation, y_validation)
plt.show()

## output

In [58]:
import csv
prediction_label = ss_label
with open('prediction.csv','w') as output:
    output.write("id,sentiment\n")
    for i in range(0,len(prediction_label)):
        output.write(str(test_data['id'].iloc[i]))
        output.write(",")
        output.write(prediction_label[i])
        output.write('\n')

In [83]:
len(y_train_s)

7971

In [67]:
X_train_s

<8079x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 11927 stored elements in Compressed Sparse Row format>